In [0]:
# !pip install pytorch_pretrained_bert
# !pip install torch

import pandas as pd
import numpy as np
import copy
import string
import torch

from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import tokenization

from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification

## Data preprocessing

In [5]:
data = pd.read_csv("./sample_0.csv").fillna(method="ffill")
data.drop(columns=['Number'], inplace=True)

data.head()

,Tag,Start,End,Content
0,ЛОК,1048,1061,Південний Буг
1,ЛОК,1073,1080,Гіпаніс
2,ЛОК,1104,1109,Дунай
3,ЛОК,1185,1192,Дністер
4,ЛОК,1358,1368,Балканська


In [0]:
# Replace tags with standard notation.
data.replace(['ЛОК', 'ОРГ', 'ПЕРС', 'РІЗН'], ['LOC', 'ORG', 'PERS', 'MISC'], inplace=True)

To mark the type of entity the BIO notation is used -so that the position of every word of a Named Entity (NE) is expressed via 'Tag' attribute's preffix. The first (and sometims only) part of entity is marked with preffix 'B' (beginning) and all its following parts - with 'I' (inner). If a word is not an entity it's marked with 'O' - meaning it's standing outside of a named entity group.

In [10]:
def is_marked(tag):
  return tag[1] == '-'


def mark_tag_position(tag, bio_mark):
  if not is_marked(tag):
    tag = bio_mark + tag
  return tag


for index, row in data.iterrows():
  content_split = row['Content'].find(' ');
  if content_split != -1:
    data.set_value(index,'Content', row['Content'][:content_split])
    for word in row['Content'].split(' ')[1:]:
      new_row = copy.copy(row)
      new_row['Content'] = word
      new_row['Tag'] = mark_tag_position(new_row['Tag'], 'I-')
      data.loc[data.shape[0]] = list(new_row)

  row['Content'] = row['Content']
  data.set_value(index, 'Tag', mark_tag_position(row['Tag'], 'B-'))

data.tail()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,Tag,Start,End,Content
35,I-LOC,2507,2525,губернію
36,I-LOC,2865,2883,держава
37,I-PERS,3766,3778,Лицар
38,I-LOC,4397,4412,Бугу
39,I-LOC,4759,4773,шляхи


## Tokenization & Input Formatting

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
# tokenizer = tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=False)  # if you have /config/vocabulary.txt locally

In [27]:
orig_tokens = data['Content']
labels = data['Tag']

bert_tokens, orig_to_tok_map = [], []

bert_tokens.append("[CLS]")
for orig_token in orig_tokens:
  orig_to_tok_map.append(len(bert_tokens))
  bert_tokens.extend(tokenizer.tokenize(orig_token))
bert_tokens.append("[SEP]")

bert_ids = tokenizer.convert_tokens_to_ids(bert_tokens)

bert_labels = []

for i in range(len(bert_ids)):
  if i in orig_to_tok_map:
    bert_labels.append(data['Tag'][orig_to_tok_map.index(i)])
  else:
    bert_labels.append('X')


# example:
print(bert_tokens[:orig_to_tok_map[3]])
print(bert_ids[:orig_to_tok_map[3]])
print(bert_labels[:orig_to_tok_map[3]])

['[CLS]', 'П', '##ів', '##ден', '##ний', 'Г', '##іп', '##ані', '##с', 'Д', '##уна', '##й']
[101, 524, 10721, 16770, 11532, 512, 30892, 41879, 10513, 513, 27063, 10384]
['X', 'B-LOC', 'X', 'X', 'X', 'B-LOC', 'X', 'X', 'X', 'B-LOC', 'X', 'X']


## Split into training set & testing set.

---



In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(bert_ids, bert_labels, random_state=7, test_size=0.2)

In [35]:
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)

Y_train = torch.tensor(Y_train)
Y_test = torch.tensor(Y_test)

ValueError: ignored

## Get the pre-trained model 

Using a pre-trained model allows us to have a decent stating point for our model (used insdead of random initial parameters), impying the task it is used for is quite similar. Here we import a multilingual BERT token-classification model (used for NER) from PyTorch lib. A pre-trained model can also be downloaded from Google AI's official repo https://github.com/google-research/bert, and then set up with config file from the archive. However the Transformers wrapper library provides a convenient importability of this model in PyTorch and TensorFlow.

In [0]:
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=4)

## Train our model

## Evaluate the model

MultiNLI

Test F1